# 远程手柄遥控
在本例中，我们将使用连接到web浏览器机器的gamepad控制器远程控制机械臂。

### 创建手柄控制器
我们要做的第一件事是创建一个' Controller'widget的实例，我们将使用它来驱动我们的Jetbot。
“Controller”小部件接受一个“index”参数，该参数指定控制器的数量。如果你有多个控制器，或者一些游戏手柄以多个控制器的形式出现，这是非常有用的。想要正确的使用你的手柄来控制
机械臂你得:
1. 打开[http://html5gamepad.com](http://html5gamepad.com)此网页.  
2. 按下你正在使用的手柄的按键
3. 记住当你按下按键后弹出的相应的索引号

接下来，我们将使用该索引创建并显示控制器。

In [1]:
import ipywidgets.widgets as widgets
controller = widgets.Controller(index=0)  #用你刚测试过正在使用的控制器的索引号替代
display(controller)

Controller()

相关模块导入

In [2]:
#函数库路径导入
import threading
import time
# 线程功能操作库
import inspect
import ctypes
# 导入机械臂对象
from Arm_Lib import Arm_Device
Arm = Arm_Device()

创建主动停止进程的方法

In [3]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

# 创建手柄遥杆控制机械臂运动的方法  
## 如果将手柄的模拟模式打开，即红灯亮起时，左边的方向键则不可以用，请使用左右摇杆和LR两边的键来控制机械臂。

## 程序功能: 
1. 左边摇杆和方向键控制一号和二号舵机，其中左右方向控制1号舵机左右运动，上下方向控制2号舵机前后运动。 
2. 右边摇杆和数字键控制五号和六号舵机，其中左右方向控制5号舵机左右翻转，上下方向控制6号舵机夹子夹紧和放松。
3. L1和L2控制3号舵机向前或者向后运行。
4. R1和R2控制4号舵机向前或者向后运行。
5. 按下SELECT键将机械臂所有舵机角度都设置为90度  


In [4]:
def Arm_Handle():
    s_time = 500
    s_step = 1
    angle_1 = angle_2 = angle_3 = angle_4 =  angle_5 = angle_6 = 90
    while 1:
        #因为摇杆手柄个别差异,所有在遥杆复位值不一定是零，所以需要以0.1作为过滤，避免误操作。
        # 2号舵机，A1上负下正
        if controller.axes[1].value <= 0.1 and controller.axes[1].value >= -0.1:
            time.sleep(.000001)
        else:
            if controller.axes[1].value > 0.1:
                angle_2 += s_step
            else:
                angle_2 -= s_step
            if angle_2 > 180:
                angle_2 = 180
            elif angle_2 < 0:
                angle_2 = 0
            Arm.Arm_serial_servo_write(2, angle_2, s_time)
            time.sleep(0.01)

        # 1号舵机，A0左负有正
        if (controller.axes[0].value <= 0.1 and controller.axes[0].value >= -0.1):
                time.sleep(.000001)
        else:
            if controller.axes[0].value > 0.1:
                angle_1 -= s_step
            else:
                angle_1 += s_step
                
            if angle_1 > 180:
                angle_1 = 180
            elif angle_1 < 0:
                angle_1 = 0
            Arm.Arm_serial_servo_write(1, angle_1, s_time)
            time.sleep(0.01)

        # 6号舵机，NUM1=B0,NUM3=B2, A2上负下正
        if controller.buttons[0].value == True:
            angle_6 += s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)
        elif controller.buttons[2].value == True:
            angle_6 -= s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)
        elif controller.axes[2].value > 0.5:
            angle_6 -= s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)
        elif controller.axes[2].value < -0.5:
            angle_6 += s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)

        # 5号舵机，NUM2=B1,NUM4=B3, A5左负有正    
        if controller.buttons[1].value == True:
            angle_5 += s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)
            time.sleep(0.01)
        elif controller.buttons[3].value == True:
            angle_5 -= s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)  
            time.sleep(0.01)
        elif controller.axes[5].value > 0.5:
            angle_5 += s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)
            time.sleep(0.01)
        elif controller.axes[5].value < -0.5:
            angle_5 -= s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)  
            time.sleep(0.01)


        # 4号舵机，R1=B5,R2=B7   
        if controller.buttons[5].value == True:
            angle_4 -= s_step
            if angle_4 > 180:
                angle_4 = 180
            elif angle_4 < 0:
                angle_4 = 0
            Arm.Arm_serial_servo_write(4, angle_4, s_time)
            time.sleep(0.01)
        elif controller.buttons[7].value == True:
            angle_4 += s_step
            if angle_4 > 180:
                angle_4 = 180
            elif angle_4 < 0:
                angle_4 = 0
            Arm.Arm_serial_servo_write(4, angle_4, s_time)  
            time.sleep(0.01)

        # 3号舵机，L1=B4,L2=B6   
        if controller.buttons[4].value == True:
            angle_3 -= s_step
            if angle_3 > 180:
                angle_3 = 180
            elif angle_3 < 0:
                angle_3 = 0
            Arm.Arm_serial_servo_write(3, angle_3, s_time)
            time.sleep(0.01)
        elif controller.buttons[6].value == True:
            angle_3 += s_step
            if angle_3 > 180:
                angle_3 = 180
            elif angle_3 < 0:
                angle_3 = 0
            Arm.Arm_serial_servo_write(3, angle_3, s_time)  
            time.sleep(0.01)
        
        # 按下选择键B8,让机械臂的舵机都设置到90度
        if controller.buttons[8].value == True:
            angle_1 = angle_2 = angle_3 = angle_4 = angle_5 = angle_6 = 90
            Arm.Arm_serial_servo_write6(90, 90, 90, 90, 90, 90, 1000)
            time.sleep(1)
            

通过运行下面单元格代码开启手柄实时控制机械臂的进程

In [5]:
thread2 = threading.Thread(target=Arm_Handle)
thread2.setDaemon(True)
thread2.start()

通过运行下面单元格代码结束手柄进程，如果出现进程启动或者结束失败的情况，请重新start一下kernel，再重头运行。

In [6]:
stop_thread(thread2)